# Calculate indicators

This notebook enables the user to use GDP, population, or other sets of indicators to calculate combined indicators such as per capita emissions or emissions / final energy use.

It should be used on data that is already pre-processed for this toolset to ensure efficiency and avoid errors.

TODO: unit handling!


In [3]:
# import modules

# system
import sys, os, re

# analytic
import pandas as pd
import numpy as np

# open cliamte data
import countrygroups

# plotting
import seaborn
import matplotlib.pyplot as plt

# global stocktake tools
import gst_tools.gst_utils as utils


In [8]:
# User options


data_set_1 = 'PRIMAP-hist_v2.0_CO2-total-excl-LU.csv'
data_set_2 = 'UN-2017-population.csv'

# Use this to generate the prefix of the output data file to include the source names of the original data. 
# The entities will automaticall be added when running the script
new_source_name = 'PRIMAP-hist_UN-2017_calc'


In [9]:
# get and clean data 

fname_in1 = os.path.join('proc-data', data_set_1)
fname_in2 = os.path.join('proc-data', data_set_2)

# read in the data
var1 = pd.read_csv(fname_in1)
var2 = pd.read_csv(fname_in2)

# make sure that the same countries and years are available
var1, var2 = utils.ensure_common_years(var1, var2)
var1, var2 = utils.ensure_common_countries(var1, var2)

# check the data format
check1 = utils.verify_data_format(var1)
check2 = utils.verify_data_format(var2)

if not check1 or not check2:
    print('One of the dataframes is not correct! Please check and try again!')
else:
    # get metadata for later use and checking
    var1_name = var1['variable'].unique()[0]
    var2_name = var2['variable'].unique()[0]

    var1_unit  = var1['unit'].unique()[0]
    var2_unit = var2['unit'].unique()[0]


Common countries are: 
['ALB', 'NIU', 'FJI', 'BLR', 'ARG', 'LAO', 'HTI', 'VEN', 'COK', 'LIE', 'SLB', 'CUB', 'TTO', 'YEM', 'GBR', 'RUS', 'THA', 'GEO', 'NOR', 'LUX', 'DJI', 'IDN', 'MWI', 'KEN', 'COL', 'AFG', 'ROU', 'AZE', 'KHM', 'SLE', 'MNG', 'BTN', 'JAM', 'ZAF', 'MHL', 'SGP', 'BFA', 'MNE', 'LVA', 'NZL', 'BOL', 'EST', 'NGA', 'TGO', 'CIV', 'NRU', 'UKR', 'FSM', 'ETH', 'PHL', 'AUT', 'MUS', 'AGO', 'NIC', 'BRB', 'COD', 'JOR', 'LBY', 'MDV', 'MLI', 'BWA', 'POL', 'SDN', 'GNB', 'GUY', 'PAK', 'GIN', 'LKA', 'KWT', 'GNQ', 'MOZ', 'HND', 'TON', 'SLV', 'HUN', 'PER', 'BRN', 'ARM', 'MEX', 'SAU', 'BEN', 'KIR', 'NLD', 'COG', 'TZA', 'STP', 'MMR', 'ZWE', 'WSM', 'NAM', 'TUR', 'KAZ', 'BDI', 'SYC', 'CAF', 'TCD', 'GRD', 'ARE', 'LSO', 'MDA', 'ISL', 'MKD', 'GTM', 'RWA', 'MRT', 'KOR', 'TUV', 'DNK', 'BIH', 'IRQ', 'MYS', 'DOM', 'CAN', 'DZA', 'GRC', 'IND', 'QAT', 'IRL', 'URY', 'SYR', 'GAB', 'SVK', 'NPL', 'FRA', 'DMA', 'PRK', 'TUN', 'SWE', 'VUT', 'SWZ', 'FIN', 'JPN', 'USA', 'CMR', 'ESP', 'CZE', 'VCT', 'AND', 'PAN', 'SU

In [10]:
# combine data...

# for all of these, it's always var1 divided by var 2 and we want to ensure that this is done on countries. 
# Everything else should be consant across the table

def prep_df_for_division(df):
    
    df = df.set_index('country')
    
    year_cols = [y for y in df[df.columns] if (re.match(r"[0-9]{4,7}$", str(y)) is not None)]
    other_cols = list(set(df.columns) - set(year_cols))
    
    df = df.drop(other_cols, axis='columns')
    
    return df
    
# strip original metadata
var1 = prep_df_for_division(var1)
var2 = prep_df_for_division(var2)

# calculate new variables
new_df = var1 / var2

# generate new metadata
new_variable_name = var1_name + '-per-' + var2_name
new_df['variable'] = new_variable_name
new_df['unit'] = var1_unit + ' / ' + var2_unit

new_df = new_df.reset_index()

# reorganise dataframe
new_df = utils.check_column_order(new_df)


In [11]:
# take a look at your new data frame

new_df

,country,unit,variable,1990,1991,1992,1993,1994,1995,1996,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,AFG,Gg / ThousandPers,CO2-total-excl-LU-per-population,0.232670,0.203176,0.114439,0.101357,0.087184,0.079534,0.072379,...,0.070288,0.091296,0.159742,0.247462,0.299967,0.417388,0.358342,0.321445,0.344954,0.358662
1,AGO,Gg / ThousandPers,CO2-total-excl-LU-per-population,0.421478,0.407060,0.401747,0.432715,0.282488,0.770902,0.715148,...,1.095625,1.195370,1.176502,1.223971,1.236674,1.246977,1.326897,1.250080,1.285267,1.360407
2,ALB,Gg / ThousandPers,CO2-total-excl-LU-per-population,1.203735,1.700540,1.036849,1.025212,1.120807,1.113709,1.086595,...,1.368548,1.494755,1.537613,1.691062,1.788796,2.036452,1.904084,1.952738,2.204894,1.966920
3,AND,Gg / ThousandPers,CO2-total-excl-LU-per-population,7.888606,7.711175,7.573699,7.462564,7.323261,7.611590,8.017402,...,7.445272,7.244536,7.118923,6.807795,6.856209,6.579026,6.611590,6.634649,6.576373,6.909016
4,ARE,Gg / ThousandPers,CO2-total-excl-LU-per-population,24.943903,25.786462,24.824610,26.501707,27.740966,25.930856,20.731072,...,28.424092,27.134047,27.559086,26.348767,23.577252,23.176775,24.043720,22.872972,28.111977,30.040521
5,ARG,Gg / ThousandPers,CO2-total-excl-LU-per-population,3.391411,3.584994,3.654715,3.605884,3.732842,3.543382,3.783207,...,4.322669,4.603427,4.655495,4.460849,4.608978,4.777122,4.703452,4.771988,4.769492,4.767634
6,ARM,Gg / ThousandPers,CO2-total-excl-LU-per-population,6.161386,5.819840,7.261510,2.964529,3.008838,3.387890,2.345169,...,1.466960,1.752438,1.980593,1.620171,1.623043,1.895269,2.186041,2.063239,2.064537,2.334630
7,ATG,Gg / ThousandPers,CO2-total-excl-LU-per-population,4.962816,4.694905,4.588832,4.604078,4.461858,4.455372,4.588248,...,5.603482,5.832722,5.893294,6.144410,6.232767,6.059403,6.117156,6.061907,6.139064,6.264824
8,AUS,Gg / ThousandPers,CO2-total-excl-LU-per-population,16.371865,16.269225,16.355991,16.395779,16.496356,16.927828,17.130330,...,19.150751,19.191600,19.023082,18.860078,18.444793,18.016175,17.877605,17.234878,16.826649,16.975107
9,AUT,Gg / ThousandPers,CO2-total-excl-LU-per-population,8.130556,8.529853,7.764081,7.741933,7.738566,8.072468,8.493535,...,9.306272,8.939117,8.874548,8.088374,8.632633,8.321647,7.983518,7.950773,7.459557,7.708566


In [12]:
## write the data to file

"""
First ensure that years, unit, 'country', and variable are all in data. If they are
can proceed to print data
"""

if 'country' not in new_df.columns or 'unit' not in new_df.columns:
    
    print('Missing required information! Please check your input data and processing!')
    
else:
    
    # define filename as composite of variable and source name
    fname_out = new_source_name + '_' + new_variable_name + '.csv' 
    fullfname_out = os.path.join('proc-data', fname_out)

    # check folder exists
    if not os.path.exists('proc-data'):
        os.makedirs('proc-data')

    # write to csv in proc data folder
    new_df.to_csv(fullfname_out, index=False)

    # celebrate success 
    print('Processed data written to file! - ' + fullfname_out)
    

Processed data written to file! - proc-data/PRIMAP-hist_UN-2017_calc_CO2-total-excl-LU-per-population.csv
